In [ ]:
!pip install accelerate
!pip install PyMuPDF
!pip install transformers

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import fitz
import re
class TR_parser:
  def __init__(self):
    # Specify the LLM model we'll be using
    model_name = "Qwen/Qwen2-0.5B-Instruct"
    #model_name ="gpt2"
    # Configure for GPU usage
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True,
    )
    # Load the tokenizer for the chosen model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # Create a pipeline object for easy text generation with the LLM
    self.pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    # Parameters to control LLM's response generation

  def query(self,messages):
    generation_args = {
        "max_new_tokens": 256,     # Maximum length of the response
        "return_full_text": False,      # Only return the generated text
    }
    """Sends a conversation history to the AI assistant and returns the answer.

    Args:
      messages (list): A list of dictionaries, each with "role" and "content" keys.

    Returns:
      str: The answer from the AI assistant.
    """

    output = self.pipe(messages, **generation_args)
    return output[0]['generated_text']
  def get_name(self,text):
    messages = [
        {"role": "system", "content": "Return the name of the person in CV data in this format  Name:  "},
        {"role": "user", "content": text}
    ]
    result = self.query(messages)
    return result
  def get_phone_number(self,text):
    messages = [
        {"role": "system", "content": "Return the phone number of the person in CV data in this format  phone number:  "},
        {"role": "user", "content": text}
    ]
    result = self.query(messages)
    return result
  def get_mail(self,text):
    messages = [
        {"role": "system", "content": "Return the mail of the person in CV data in this format  phone mail:  "},
        {"role": "user", "content": text}
    ]
    result = self.query(messages)
    return result
  def get_about(self,text):
    messages = [
        {"role": "system", "content": "generat a Summary profil  for the person in CV data,in this format  Summary:  "},
        {"role": "user", "content": text}
    ]
    result = self.query(messages)
    return result
  def get_hobbies(self,text):
    messages = [
        {"role": "system", "content": "get hobbies of the person in CV data,in this list format  Hobbies:[]  "},
        {"role": "user", "content": text}
    ]
    result = self.query(messages)
    return result
  def get_education(self,text):
    messages = [
        {"role": "system", "content": "get education degrees of the person in CV data,in this list format  Education:[]  "},
        {"role": "user", "content": text}
    ]
    result = self.query(messages)
    return result
  def get_experience(self,text):
    messages = [
        {"role": "system", "content": "get experiences  of the person in CV data,in this list format  experiences:[]  "},
        {"role": "user", "content": text}
    ]
    result = self.query(messages)
    return result
  def get_languages(self,text):
    messages = [
        {"role": "system", "content": "get languages  of the person in CV data,in this format  languages:[]  "},
        {"role": "user", "content": text}
    ]
    result = self.query(messages)
    return result
  def extract_text_from_pdf(self,pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text
  def clean_extracted_text(self,text):
      # Supprimer les espaces multiples
      text = re.sub(r'\s+', ' ', text)
      # Supprimer les nouvelles lignes multiples
      text = re.sub(r'\n+', '\n', text)
      return text

c:\Users\161\PythonRepository\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class personCV:
  def __init__(self,parser,cvtext,cvpath):
    #if type(parser)==TR_parser:
      self.cv_path=cvpath
      name=parser.get_name(cvtext)
      self.name=name.split(sep=":")[1]
      phone_number=parser.get_phone_number(cvtext)
      self.phonenumber=phone_number.split(sep=":")[1]
      mail=parser.get_mail(cvtext)
      self.mail=mail.split(sep=":")[1]
      Summary=parser.get_about(cvtext)
      self.summary=Summary
      hobbies=parser.get_hobbies(cvtext)
      self.hobbies=hobbies
      education=parser.get_education(cvtext)
      self.education=education
      experience=parser.get_experience(cvtext)
      self.experience=experience
      languages=parser.get_languages(cvtext)
      self.languages=languages

    #else:
    #  self.name=None
    #  self.phonenumber=None



  def get_name(self):
    return self.name
  def get_phone_number(self):
    return self.phonenumber



In [4]:
trp=TR_parser()


c:\Users\161\PythonRepository\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\161\.cache\huggingface\hub\models--Qwen--Qwen2-0.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned 

In [ ]:
import os




dossier_pdf='/content/drive/MyDrive/cv'
for filename in os.listdir(dossier_pdf):
    if filename.endswith('.pdf'):
        filename='Aymen__Hmid_CV.pdf'
        resume_path = os.path.join(dossier_pdf, filename)
        text = trp.extract_text_from_pdf(resume_path)
        text = trp.clean_extracted_text(text)
        person=personCV(trp,text,resume_path)
        print(person.get_name())
        print(person.get_phone_number())
        print(person.get_mail())
        print
        break
